In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

In [2]:
#! jupyter nbextension enable --py gmaps

In [3]:
#! pip install gmaps

In [4]:
# Output File (CSV)
exported_cities_data = ('../output_data/cities_data_rows_fixed')
Weather_Results_df = pd.read_csv(exported_cities_data)
Weather_Results_df.head(100)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,punta arenas,-53.1500,-70.9167,53.11,66,100,20.83,CL,1647907311
1,lorengau,-2.0226,147.2712,84.36,69,74,4.00,PG,1647907311
2,new norfolk,-42.7826,147.0587,59.04,77,100,0.81,AU,1647907312
3,qaanaaq,77.4840,-69.3632,-5.64,88,100,5.75,GL,1647907312
4,kapaa,22.0752,-159.3190,80.28,65,72,1.99,US,1647907313
...,...,...,...,...,...,...,...,...,...
95,sao filipe,59.7035,30.7861,33.73,91,0,6.67,RU,1647907349
96,henties bay,72.7868,-56.1549,-18.65,65,93,5.61,GL,1647907349
97,mahebourg,39.8161,109.9776,33.37,50,100,5.70,CN,1647907350
98,general roca,58.6700,52.1932,15.64,99,94,4.47,RU,1647907350


In [5]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [6]:
#Store 'Lat' and 'Lng' into  locations 
Locations = Weather_Results_df[['Lat', 'Lng']].astype(float)

#Store 'Humidity'
Humidity = Weather_Results_df['Humidity'].astype(float)

In [7]:
#Create a figure
fig = gmaps.figure(zoom_level = 2, center = (25, -5))

#Create a humidity layer
Humidity_layer = gmaps.heatmap_layer(Locations, weights=Humidity, dissipating=False, max_intensity=170, point_radius = 4)

#Add the humidity layer to the figure
fig.add_layer(Humidity_layer)

#Show the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#Creating Ideal Vacation DataFrame (filtered by set preferences)
Ideal_Vacation_df=Weather_Results_df.loc[((Weather_Results_df["Max Temp"]<80) & (Weather_Results_df["Max Temp"]>70))]
Ideal_Vacation_df=Weather_Results_df.loc[Weather_Results_df["Wind Speed"]<10]
Ideal_Vacation_df=Weather_Results_df.loc[Weather_Results_df["Cloudiness"]==0]
Ideal_Vacation_df = Ideal_Vacation_df.dropna(how='any')
Ideal_Vacation_df.head(10)



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,rikitea,-29.9533,-71.3436,65.34,71,0,4.83,CL,1647907316
33,bredasdorp,-33.6500,115.3333,59.58,67,0,11.99,AU,1647907324
35,yar-sale,-34.0033,24.7314,63.88,90,0,4.18,ZA,1647907325
49,bokspits,-27.1833,151.2667,76.17,50,0,7.23,AU,1647907331
59,tuktoyaktuk,-33.9258,18.4232,65.46,82,0,14.97,ZA,1647907103
74,nhulunbuy,27.9659,-82.8001,76.41,52,0,15.01,US,1647907241
79,fortuna,41.7075,-86.8950,64.72,39,0,6.91,US,1647907207
85,flinders,38.8876,-76.5119,60.26,49,0,6.91,US,1647907344
95,sao filipe,59.7035,30.7861,33.73,91,0,6.67,RU,1647907349
101,kachikau,-33.5906,26.8910,70.20,87,0,11.18,ZA,1647907351


In [9]:
hotel_df = Ideal_Vacation_df
#Adding 'Hotel Name' column to my hotel_df DataFrame
hotel_df["Hotel Name"]=""
hotel_df["rating"]=""
#hotel_df["vicinity"]=""
hotel_df=hotel_df.dropna()
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,rating
12,rikitea,-29.9533,-71.3436,65.34,71,0,4.83,CL,1647907316,,
33,bredasdorp,-33.6500,115.3333,59.58,67,0,11.99,AU,1647907324,,
35,yar-sale,-34.0033,24.7314,63.88,90,0,4.18,ZA,1647907325,,
49,bokspits,-27.1833,151.2667,76.17,50,0,7.23,AU,1647907331,,
59,tuktoyaktuk,-33.9258,18.4232,65.46,82,0,14.97,ZA,1647907103,,


In [10]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" with lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.head(10)        

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -29.985466,
                    "lng": -71.36058799999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.98421322010728,
                        "lng": -71.35915012010727
                    },
                    "southwest": {
                        "lat": -29.98691287989272,
                        "lng": -71.36184977989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "apart hotel la herradura",
            "photos": [
                {
                 

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.6466789,
                    "lng": 115.3453292
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.64530872010727,
                        "lng": 115.3467842798927
                    },
                    "southwest": {
                        "lat": -33.64800837989272,
                        "lng": 115.3440846201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Esplanade Hotel",
            "opening_hours": {
                "open_now": true
            

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -34.0309629,
                    "lng": 24.7679228
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.02964552010727,
                        "lng": 24.76926122989272
                    },
                    "southwest": {
                        "lat": -34.03234517989272,
                        "lng": 24.76656157010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Humansdorp Boutique Hotel",
            "opening_hours": {
                "open_now": false
  

{
    "html_attributions": [],
    "next_page_token": "Aap_uECG14jfDUmXPA_X36_9aaEpSkA0hA1tYPVvx3yl_PWbH8DPjOicymFjkTi3oepiL4gvARlv1nYkOx7peoykYsI9voZ-SJbPUEVqiJMhzQ-BYmo-l6VvzEoy21kW3sZRcJMVzRXq80Cfxj3-X9mT66BUipO031h6IunsSqcL7ldNmv5lGGn3NOaMV8kHAC2RcHdgOAfCD7dq5iF94_gK3bsvNmZwT_wbrAG5RbJfBYUDB8ZuODebRMdCnreezCH_Eh1AEEs27uE5wXWh0U5nBjm3n0wuCQkIRsViZcbFBM8c24Db2R2ZowYdJejQJt_PCvsqt47BM9b_JZbk7L-iw8BAhLK1cFyHFizEhsHFotSFwf-vjDkvf1ynQZo2sDijVKw4-kKpGoAx6XDnjWrWtY2U0qmEigKPe4vnBqrdQlAC25XRaMVn7LuEn5UC6pqDLQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.9246054,
                    "lng": 18.3835562
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.92318517010728,
                        "lng": 18.38476322989272
                    },
                    "southwest": {
                        "lat": -33.9258

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAO1o3oQWNQcILSwIeeofZkqgbVzcwyIQrcdCKW0K6fbfqgMgRp3J9wK_q_nUTu6M2MJPStAia_v_2ZjljmdG0NPk6X7lJVN0Wtr-9rRMi3MzwrXs6G0WBWSrQcedjs7rJF2hXHPLMVVyi7KK0OTWSofGDCtDERF6nE4lsT5fgidF-ij97C7k898ZsIxJ1Hbbu6oIikR9lfzuf7FiJGKkEo6YUjbCP4ooyy-Yo5_DUZfoGGXDmj2DC6bslSAe4U55j9_8EmTYSne7jcydrnYcRvM75JkInF2oJWskzU1JSzeFYU0kW6pzZ38XopDWaVFS36qDU5FlFLjUX1C3KFotFrtfATthzQjP8xdNoE10lMjEVlPkYWKSpMeLoDVrN-eX8uTNg0hyxESAlAW1SRgowRCOsA1Oxo5jUsJyDtoyL999v7nBFirfoEfHsYtzJv_Q",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 27.9678274,
                    "lng": -82.8282988
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.96926807989272,
                        "lng": -82.82677082010727
                    },
                    "southwest": {
                        "lat": 27.96656

...

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 42.1976465,
                    "lng": -70.7260621
                },
                "viewport": {
                    "northeast": {
                        "lat": 42.19899767989271,
                        "lng": -70.72483342010729
                    },
                    "southwest": {
                        "lat": 42.19629802010727,
                        "lng": -70.72753307989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "The Inn at Scituate Harbor",
            "opening_hours": {
                "open_now": true
  

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 21.4595508,
                    "lng": -71.14743039999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 21.46089267989272,
                        "lng": -71.14604782010728
                    },
                    "southwest": {
                        "lat": 21.45819302010728,
                        "lng": -71.14874747989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pin

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -12.7642378,
                    "lng": 45.236721
                },
                "viewport": {
                    "northeast": {
                        "lat": -12.76288797010728,
                        "lng": 45.23807082989272
                    },
                    "southwest": {
                        "lat": -12.76558762989272,
                        "lng": 45.23537117010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "H\u00f4tel Hamaha beach",
            "opening_hours": {
                "open_now": true
      

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -31.632286,
                    "lng": -71.1663641
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.63090547010728,
                        "lng": -71.16503702010728
                    },
                    "southwest": {
                        "lat": -31.63360512989273,
                        "lng": -71.16773667989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hotel Diaguitas Illapel",
            "photos": [
                {
                    "heig

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,rating
12,rikitea,-29.9533,-71.3436,65.34,71,0,4.83,CL,1647907316,apart hotel la herradura,4.5
33,bredasdorp,-33.6500,115.3333,59.58,67,0,11.99,AU,1647907324,Esplanade Hotel,4.2
35,yar-sale,-34.0033,24.7314,63.88,90,0,4.18,ZA,1647907325,Humansdorp Boutique Hotel,3.5
49,bokspits,-27.1833,151.2667,76.17,50,0,7.23,AU,1647907331,Windsor Hotel Dalby,4.1
59,tuktoyaktuk,-33.9258,18.4232,65.46,82,0,14.97,ZA,1647907103,Grande Kloof Boutique Hotel,4
74,nhulunbuy,27.9659,-82.8001,76.41,52,0,15.01,US,1647907241,"Holiday Inn & Suites Clearwater Beach, an IHG ...",4.2
79,fortuna,41.7075,-86.8950,64.72,39,0,6.91,US,1647907207,Hampton Inn & Suites Michigan City,4.2
85,flinders,38.8876,-76.5119,60.26,49,0,6.91,US,1647907344,,
95,sao filipe,59.7035,30.7861,33.73,91,0,6.67,RU,1647907349,,
101,kachikau,-33.5906,26.8910,70.20,87,0,11.18,ZA,1647907351,Royal St Andrews Hotel,4.5


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
#Create a figure
fig = gmaps.figure(zoom_level = 2, center = (25, -5))

#Create Hotel marker layer
Hotel_layer = gmaps.marker_layer(
    locations, info_box_content = hotel_info
)
#Add the 'humidity' and 'hotel' layers to the figure
fig.add_layer(Humidity_layer)
fig.add_layer(Hotel_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))